<a href="https://colab.research.google.com/github/kaz-kobayashi/2019rinko/blob/master/190606-13_rinko_integer_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ナップサック問題

重さが$w_j$(kg)で価格が$c_j$（千円）である品物$j$がある ($j=1,2,...,5$)．これらを15kgの容量のナップサックに，選んで詰めてもって行こう．品物の総価格が最大になるにはどれを選んだらよいでしょうか？

|    | 品物1 |  品物2 | 品物3 | 品物4 | 品物5 |
| :--- | :---: | ---: |---: |---: |---: |
| 価格$c_j$（千円） | 50 | 40 | 10 | 70  |55 |
| 重量$w_j$(kg) | 7 | 5 | 1  | 9 | 6 |

品物ごとに0-1変数$x_j \in \{0,1\}$を割り当て，その変数が0ならばナップサックに入れない，１ならば入れると考えます．目的関数は，価格$c_j$に$x_j$を掛けて，全ての品物について足し合わせたものである．制約条件は，総重量が15kgを超えないということ．まとめると以下のような最適化問題となります．

$
\max 50x_1 + 40x_2 + 10x_3 + 70x_4 + 55x_5 
$

$
\text{s.t.  }  7x_1 + 5x_2 + x_3 + 9x_4 + 6x_5  \leq 15
$

$
\quad \quad x_j \in \{0,1\} j=1,2,...,5
$


## 準備
PuLPを使うために，まずはインストールしましょう．

In [0]:
pip install pulp

## 演習1-1

この問題を，PuLPを使って(0-1)整数線形最適化問題として定式化して解きましょう．
BeProud斎藤さんによるPuLPのチートシートがあります．scrapboxの19-06-06輪講ページを参照ください．

ヒント　0-1変数を定義するときは，LpVariable()で変数を生成するときに，型としてLpBinaryを指定します． LpVariable(......,cat=LpBinary)


In [0]:
from pulp import *
x1=LpVariable('x1',cat=LpBinary)
x2=LpVariable('x2',cat=LpBinary)
x3=LpVariable('x3',cat=LpBinary)
x4=LpVariable('x4',cat=LpBinary)
x5=LpVariable('x5',cat=LpBinary)
knapsack=LpProblem('Knapsack',sense=LpMaximize)
knapsack+=50*x1+40*x2+10*x3+70*x4+55*x5
knapsack+=7*x1+5*x2+1*x3+9*x4+6*x5<=15

knapsack.solve()

print("status=",LpStatus[knapsack.status]," optimval value=",knapsack.objective)

for v in knapsack.variables():
    print(v.name,"=",value(v))
 
print("total value=",value(knapsack.objective))


## 演習1-2

品物の数を10に増やして，それぞれの価格と重量を設定し，演習１で作成したプログラムで解いてみましょう．


In [0]:
nItems=10
#↓アイテムを表すリスト[0,1,2,...,9]を作る
items=list(range(nItems))
print("items:",items)

#↓x0,x1,...,x9と，10個の変数の名前を作る
x_names=['x'+str(i) for i in items]
print("x_names:",x_names)

#↓x0,x1...,x9という10個の名前で，PuLPで使う変数を生成する:x[0],x[1],...,x[9]
x=[LpVariable(i,cat=LpBinary) for i in x_names]
print("x:",x)

#↓価値c[0],c[1],...,c[9]と重量w[0],w[1],...w[9]を生成する．また，ナップサックの容量Bを設定する
c=[50,40,10,70,55,32,8,29,10,49]
w=[5,3,8,2,7,4,10,2,3,4]
B=30

knapsack2=LpProblem('Knapsack2',sense=LpMaximize)


次に目的関数を設定しますが，lpSum()を使うと便利です．lpSum()の引数には，目的関数の各項を要素とするリストを渡します．この問題では，  [c[0]*x[0], c[1]*x[1], ...,c[9]*x[9]] （[ 50*x0, 40*x1, 10*x2,...,49x9]） となっているリストを渡すことになります．

これには，内包表記を使って，[  c[i]*x[i] for i in items ] と書けばいいです．

In [0]:
obj_term = [ c[i]*x[i] for i in items ]
for i in obj_term:
    print(i)

In [0]:
#↓目的関数c[0]*x[0]+c[1]*x[1]+...+c[9]*x[9]を問題に設定する
knapsack2+=lpSum([c[i]*x[i] for i in items])

#制約式を追加する
knapsack2+=lpSum([w[i]*x[i] for i in items])<=B

print(knapsack2)

#解く
knapsack2.solve()

print(LpStatus[knapsack2.status])

for v in knapsack2.variables():
    print(v.name,"=",value(v))
 
print("total value=",value(knapsack2.objective))


## 演習1-3

品物の数を$n$とすることにして，$n$個の品物の価格と重量を乱数で設定するプログラムを作成しましょう．そして，$n=30,40,50$に対して価格と重量を実際に生成し，演習１で作成したプログラムでナップサック問題を解いてみましょう．その際，各$n$での実行時間を計測しましょう．


ヒント　整数の乱数を使って，価値のリストcと重量のリストwを生成しましょう．整数の乱数を生成するには，numpyの中のrandomを使います．

In [0]:
from numpy.random import *
#0から99までの整数の乱数を20個生成
randint(10,100,20)

# 集合分割問題

集合分割問題は，与えられた集合を，その部分集合に分割する問題です．候補の部分集合$j$にはそれぞれ費用$c_j$が与えられていますが，採用する部分集合の費用の和を最小にするように部分集合を選ぶことがこの問題の目的です．分割したい元の集合の要素を$1,,2,...m$，候補とする部分集合を$1,2,...,N$，部分集合$j$を採用するとき1，それ以外のとき0をとる0-1変数を$x_j$，元の集合の要素$i$が部分集合$j$に含まれるとき1，それ以外のとき0として$a_{ij}$を定めると，集合分割問題は次の整数線形最適化問題として定式化できます．

$
\displaystyle \min \sum_{j=1}^N c_j x_j 
$

$
\displaystyle  \text{s.t. } \sum_{j=1}^N a_{ij} x_j = 1 \quad \forall i =1,2,...m
$

$
\displaystyle  \quad \quad x_j \in \{0,1\} \quad \forall  j=1,2,...N
$



##演習2-1

集合分割問題を整数線形最適化問題として定式化したものを PuLPで実装しましょう．ただし，データは次のものとします．$a_{ij}$の値は，行列$A$の$(i,j)$成分とします．

$m=5, n=8, c=[8,6,10,12,7,6,15,9]$

$
A=
\begin{bmatrix}
1 & 0 & 0 & 1 & 0 &0 &0 & 0\\
0 & 1 & 1 & 1 & 0 &1 &1 & 0\\
0 & 0 & 0 & 0 & 1 &0 &1 & 0\\
1 & 1 & 0 & 0 & 0 &1 &1 & 0 \\
0 & 0 & 1 & 1 & 0 &1 &0 & 1
\end{bmatrix}
$


ヒント，
$a_{ij}$は，numpy.arrayを使って定義すると便利です．scrapboxの記事をご覧ください．a=numpy.array([[１行目],[2行目],[3行目]...])などと定義して，a[i]でi+1行目にアクセスできるようにしておくと制約式の定義が楽です．

In [0]:
import numpy as np

In [0]:
A=np.array([ [1,0,0,1,0,0,0,0], [0,1,1,1,0,1,1,0], [0,0,0,0,1,0,1,0], [1,1,0,0,0,1,1,0], [0,0,1,1,0,1,0,1] ])
print(A)

これで，A[i]とすると行列Aのi行目にアクセスできるようになりました．

問題を定義してから，目的関数を設定するまでは，次の通りです．

In [0]:
m=5
nSubsets=8
subsets=list(range(nSubsets))
print("subsets:",subsets)

#↓x0,x1,...,x9と，10個の変数の名前を作る
x_names=['x'+str(i) for i in subsets]

#各部分集合に対して変数を定義したいので，それらの名前を用意しておく
col_names=['x'+str(i) for i in range(n)]
print("col_names:",col_names)
#変数を生成する
x=[LpVariable(i,cat=LpBinary) for i in col_names]

c=[8,6,10,12,7,6,15,9]
print(x,c)
sp=LpProblem('set partitioning',sense=LpMinimize)
sp+=lpSum( [c[i]*x[i] for i in subsets ] )
print(sp)

ここまでで，目的関数が設定できました．あとは，$m$本の制約式を追加すれば定式化の完成です．挑戦してみてください．

## 結婚式の席決め問題

https://pythonhosted.org/PuLP/CaseStudies/a_set_partitioning_problem.html

### 集合分割問題としての定式化
結婚式のゲストを，４人ずつのテーブルに分けなければなりません（４人より少ないテーブルがあってもよいです）．そのとき，分け方がゲストの皆さんにとって出来るだけハッピーなものでなければなりません．この問題は，結婚式のゲストを集合の要素，テーブルへの分け方を候補となる部分集合とすると，集合分割問題として定式化されます．

### 集合と部分集合
結婚式のゲストを要素する集合をguests, テーブルでの席の候補を要素とする部分集合の集合を，possible_tablesとします．guestsは，A,B,...Rとしましょう．


guests =  'A B C D E F G I J K L M N O P Q R'.split()

これを実行して， guestsはAからRまでのアルファベットを要素とするリストとなっていることを確認してください．


In [60]:
guests =  'A B C D E F G I J K L M N O P Q R'.split()
print("guests:",guests)

guests: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R']


### データのプログラムでの生成

ここでは，演習2-1のときとは違って，問題例を定めるデータ（パラメータ）$a_{ij}, c_j$もプログラムで生成してみましょう．

まず，ゲストの座席への分け方を全て列挙しましょう．集合guestsの要素の組合せで，４人以下のものを列挙するには，pulp.allcombinations()が便利です．テーブル最大人数をmax_table_size=4としておきましょう．ゲストの４人以下の組合せを，possible_tablesとして記憶することにします．

allcombinations()は，１番目の引数として与えられたリスト(guests)の要素から，２番目の引数として与えられた整数(max_table_size)の要素数の部分集合を全て生成します．

In [64]:
max_table_size=4
possible_tables=[tuple(c) for c in pulp.allcombinations(guests,max_table_size)]
print(len(possible_tables))

3213


possible_tablesの要素数は，3213個になります．ですので，集合分割問題の整数線形最適化の定式化での$n$は，この問題では$n=3213$になります．

次に，possible_tablesの各要素に対して，それを採用するか否かを表す0-1変数を定義しましょう．これには，LpVariable.dicts()を使うと便利です．




In [0]:
x=pulp.LpVariable.dicts('table',possible_tables,cat=LpBinary)
print(len(x))

これで，3213個の0-1変数が定義されました．

次に，各部分集合のコストを定義します．3213個のそれぞれに値を指定してもいいのですが，大変なので今回は簡単な関数でコストを決めることにします。


In [0]:
def happiness(table):
    return abs(ord(table[0]) - ord(table[-1]))

この関数happiness(table)は，引数tableにテーブル座席（3213個の候補の一つ）を与えると，その座席割り当ての幸福度を計算するものです．happinesss(table)の引数tableとなるのは，('A', 'B', 'E')などという，ゲストの席割り候補です．この席割り候補は，文字列を要素とするタプルです．いま，

In [0]:
table=('A', 'B', 'E')

だとしましょう．これをhappiness()の引数として渡すと，どういう計算が行われるでしょうか．まず，table[0]は，最初の要素’A'を表します． そして，table[-1]は最後の要素を表します．ord()という関数は，引数のアスキーコードを得るものです．細かいことは抜きにすると，ord()は，文字に対して数値を返すものだと思ってください．アルファベットに対しては，Aから順に，A,B,と１ずつ違う値が付いています．試しに，guestsの各要素に対してordの値を表示してみます．

In [0]:
for g in guests:
    print(ord(g))

In [0]:
print(table[0],ord(table[0]))
print(table[-1],ord(table[-1]))
print(ord('A'))
print(ord('B'))
print(ord('D'))
print(ord('Z'))

### 整数線形最適化問題の生成

ここまでできたので，最適化問題のオブジェクトを生成しましょう。


In [0]:
seating_model = pulp.LpProblem("Wedding Seating Model",pulp.LpMinimize)

### 目的関数
これに，目的関数を定義します．


In [0]:
seating_model += sum([happiness(table)*x[table]  for table in possible_tables])

possible_tablesのそれぞれの要素tableに対して，happiness関数で幸福度を計算し，対応する変数x[table]にかけたものを全て足しています．

###  制約式

制約式は2つです．

1つ目の制約は，テーブルの最大数です．今回は，テーブルは5つまでとして，max_tables=5によって最大数を表すことにしましょう． 


In [0]:
max_tables=5

使われるテーブルの総数は，1になる変数の数なので，1つ目の制約式は，次で表せます．

In [0]:
seating_model+=sum([ x[table] for table in possible_tables]) <= max_tables, "Maximum_number_of_tables"

In [0]:
print(seating_model)

2つ目の制約は，各ゲストはちょうど1つの席に割り当てらる，という制約です．これは，1人のゲストに対して1つの制約式で表せますから，guestsの要素に対するfor文で書くことができます．制約を，演習2-1のように行列Aで表したとすると，各行（縦）が，各ゲストに対応し，各列（横）が，テーブル割り当ての各候補に対応します．ですので，各ゲストに対する制約であるこの２番目の制約は，行列の値を横にみていくことになります．具体的には，行列の第i行目を横に見ていって，1がある列のうちの1つだけが１になっている，という制約を書けばよいことになります．

割り当て候補tableに対しては，0-1変数x[table]が定義されていることに注意してください．すると，ゲストguestがいずれかのテーブルにちょうど1回割り当てられる，という制約は，割り当て候補tableの中でguestを含んでいる場合だけ，x[table]を足していったものが，ちょうど１になっている，という制約式で表すことができます．x[table]を全て足す，という命令は，

lpSum( [ x[table] for table in possible_tables ] )

と表せることを思い出してください．全て足してしまうと，guestが含まれない割り当て候補のものを足されてしまうので，guestが含まれるtableに対する変数x[table]だけを足すために，内包表記で条件をつけます．

lpSum( [ x[table] for table in possible_tables if guest in table] )


In [0]:
for guest in guests:
    seating_model+=lpSum([x[table] for table in possible_tables if guest in table]) ==1, "Must_seat_%s"%guest

In [0]:
print(seating_model)

こうして，席決め問題に対する整数線形最適化問題を書くことができました．


## 演習3-1

結婚式の席決め問題を，上の手順によりPuLPを用いて解いてみましょう．



## 演習3-2

幸福度を定める関数happiness()を変更して，異なる結果が得られるようにしましょう．
